In [2]:
from pathlib import Path
import cv2, json, numpy as np
from PIL import Image
from tqdm import tqdm
import torch
torch.cuda.is_available()

True

### 路径说明
```
FS2K
├─data # FS2K数据集位置
│  └─FS2K
│      ├─photo
│      │  ├─photo1
│      │  ├─photo2
│      │  └─photo3
│      └─sketch
│          ├─sketch1
│          ├─sketch2
│          └─sketch3
├─save # 模型保存位置
└─FS2K.ipynb # 代码
```

In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 定义Dataset
class DS(Dataset):
    def __init__(s, dataD, mode='train'):
        super().__init__()
        s.dataD = dataD
        s.mode = mode
        s.xtf = transforms.Compose([
            transforms.Resize((250,250)),
            transforms.ToTensor(),
        ])
        s.ytf = transforms.Compose([
            torch.tensor,
        ])
        s.data = s.read()
    
    def read(s):
        D = s.dataD
        jp = D / f'anno_{s.mode}.json'
        with jp.open('r', encoding='utf-8')as f:
            annos = json.load(f)
        return annos

    def __getitem__(s, i):
        a = s.data[i]
        imgP = s.dataD/ f"photo/{a['image_name']}.jpg"
        img = s.xtf(Image.open(imgP.as_posix()))
        colors = a['lip_color']+a['eye_color']
        attrs = list(map(int,[a['hair'],a['hair_color'],a['gender'],a['earring'],a['smile'],a['frontal_face']]))
        return img, s.ytf(colors), torch.tensor(attrs, dtype=int)

    def __len__(s):
        return len(s.data)

rootdir = '/content/drive/MyDrive/DeepLearningHW/'
# 实例化Dataset
dataD = Path(rootdir + 'data/FS2K')
train_ds = DS(dataD)
val_ds = DS(dataD, 'test')

# 创建Dataloader
train_dl = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=0)
val_dl = DataLoader(val_ds, batch_size=16, shuffle=True, num_workers=0)

### Dataset说明
每一个样本包含三个变量img, colors, attrs  
img为tensor图片  
colors为一个6元素的float类型一维数组, 前三个表示嘴唇颜色lip_color, 后三个表示眼睛颜色eye_color  
attrs为6元素的整型一位数组, 分别为hair, hair_color, gender, earring, smile, frontal_face

In [ ]:
x, colors, attrs = train_ds[0]
colors, attrs

(tensor([156.9775,  82.5112,  79.0000, 118.6518,  72.2589,  69.5982]),
 tensor([0, 2, 0, 1, 1, 1]))

In [ ]:
{
    "image_name": "photo1/image0110",

    "skin_patch": [163, 139],
    # a point of face region.

    "lip_color": [156.97750511247443, 82.51124744376278, 79.0],
    # the mean RGB value of lip area.

    "eye_color": [118.65178571428571, 72.25892857142857, 69.59821428571429],
    # the mean RGB value of eye area.

    "hair": 0,
    # 0: with hair, 1: without hair.

    "hair_color": 2,
    # 0: brown, 1: black, 2: red, 3: no-hair, 4: golden.

    "gender": 0,
    # 0: male, 1: female.

    "earring": 1,
    # 0: with earring, 1: without earring.

    "smile": 1,
    # 0: with smile, 1: without smile.

    "frontal_face": 1,
    # 0: head rotates within 30 degrees, 1: > 30 degrees

    "style": 0
    # Style = one of {0, 1, 2}, please refer to the sketch samples.
}

{'image_name': 'photo1/image0110',
 'skin_patch': [163, 139],
 'lip_color': [156.97750511247443, 82.51124744376278, 79.0],
 'eye_color': [118.65178571428571, 72.25892857142857, 69.59821428571429],
 'hair': 0,
 'hair_color': 2,
 'gender': 0,
 'earring': 1,
 'smile': 1,
 'frontal_face': 1,
 'style': 0}

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class _Hswish(nn.Module):
    def __init__(self, inplace=True):
        super(_Hswish, self).__init__()
        self.relu6 = nn.ReLU6(inplace)

    def forward(self, x):
        return x * self.relu6(x + 3.) / 6.


class _Hsigmoid(nn.Module):
    def __init__(self, inplace=True):
        super(_Hsigmoid, self).__init__()
        self.relu6 = nn.ReLU6(inplace)

    def forward(self, x):
        return self.relu6(x + 3.) / 6.


class _ConvBNHswish(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0,
                 dilation=1, groups=1, norm_layer=nn.BatchNorm2d, **kwargs):
        super(_ConvBNHswish, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias=False)
        self.bn = norm_layer(out_channels)
        self.act = _Hswish(True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.act(x)
        return x


class SEModule(nn.Module):
    def __init__(self, in_channels, reduction=4):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            _Hsigmoid(True)
        )

    def forward(self, x):
        n, c, _, _ = x.size()
        out = self.avg_pool(x).view(n, c)
        out = self.fc(out).view(n, c, 1, 1)
        return x * out.expand_as(x)


class Identity(nn.Module):
    def __init__(self, in_channels):
        super(Identity, self).__init__()

    def forward(self, x):
        return x


class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels, exp_size, kernel_size, stride, dilation=1, se=False, nl='RE',
                 norm_layer=nn.BatchNorm2d, **kwargs):
        super(Bottleneck, self).__init__()
        assert stride in [1, 2]
        self.use_res_connect = stride == 1 and in_channels == out_channels
        if nl == 'HS':
            act = _Hswish
        else:
            act = nn.ReLU
        if se:
            SELayer = SEModule
        else:
            SELayer = Identity

        self.conv = nn.Sequential(
            # pw
            nn.Conv2d(in_channels, exp_size, 1, bias=False),
            norm_layer(exp_size),
            act(True),
            # dw
            nn.Conv2d(exp_size, exp_size, kernel_size, stride, (kernel_size - 1) // 2 * dilation,
                      dilation, groups=exp_size, bias=False),
            norm_layer(exp_size),
            SELayer(exp_size),
            act(True),
            # pw-linear
            nn.Conv2d(exp_size, out_channels, 1, bias=False),
            norm_layer(out_channels)
        )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV3(nn.Module):
    def __init__(self, nclass=1000, mode='large', width_mult=1.0, dilated=False, norm_layer=nn.BatchNorm2d):
        super(MobileNetV3, self).__init__()
        if mode == 'large':
            layer1_setting = [
                # k, exp_size, c, se, nl, s
                [3, 16, 16, False, 'RE', 1],
                [3, 64, 24, False, 'RE', 2],
                [3, 72, 24, False, 'RE', 1], ]
            layer2_setting = [
                [5, 72, 40, True, 'RE', 2],
                [5, 120, 40, True, 'RE', 1],
                [5, 120, 40, True, 'RE', 1], ]
            layer3_setting = [
                [3, 240, 80, False, 'HS', 2],
                [3, 200, 80, False, 'HS', 1],
                [3, 184, 80, False, 'HS', 1],
                [3, 184, 80, False, 'HS', 1],
                [3, 480, 112, True, 'HS', 1],
                [3, 672, 112, True, 'HS', 1],
                [5, 672, 112, True, 'HS', 1], ]
            layer4_setting = [
                [5, 672, 160, True, 'HS', 2],
                [5, 960, 160, True, 'HS', 1], ]
        elif mode == 'small':
            layer1_setting = [
                # k, exp_size, c, se, nl, s
                [3, 16, 16, True, 'RE', 2], ]
            layer2_setting = [
                [3, 72, 24, False, 'RE', 2],
                [3, 88, 24, False, 'RE', 1], ]
            layer3_setting = [
                [5, 96, 40, True, 'HS', 2],
                [5, 240, 40, True, 'HS', 1],
                [5, 240, 40, True, 'HS', 1],
                [5, 120, 48, True, 'HS', 1],
                [5, 144, 48, True, 'HS', 1], ]
            layer4_setting = [
                [5, 288, 96, True, 'HS', 2],
                [5, 576, 96, True, 'HS', 1],
                [5, 576, 96, True, 'HS', 1], ]
        else:
            raise ValueError('Unknown mode.')

        # building first layer
        self.in_channels = int(16 * width_mult) if width_mult > 1.0 else 16
        self.conv1 = _ConvBNHswish(3, self.in_channels, 3, 2, 1, norm_layer=norm_layer)

        # building bottleneck blocks
        self.layer1 = self._make_layer(Bottleneck, layer1_setting,
                                       width_mult, norm_layer=norm_layer)
        self.layer2 = self._make_layer(Bottleneck, layer2_setting,
                                       width_mult, norm_layer=norm_layer)
        self.layer3 = self._make_layer(Bottleneck, layer3_setting,
                                       width_mult, norm_layer=norm_layer)
        if dilated:
            self.layer4 = self._make_layer(Bottleneck, layer4_setting,
                                           width_mult, dilation=2, norm_layer=norm_layer)
        else:
            self.layer4 = self._make_layer(Bottleneck, layer4_setting,
                                           width_mult, norm_layer=norm_layer)

        # building last several layers
        classifier = list()
        if mode == 'large':
            last_bneck_channels = int(960 * width_mult) if width_mult > 1.0 else 960
            self.layer5 = _ConvBNHswish(self.in_channels, last_bneck_channels, 1, norm_layer=norm_layer)
            classifier.append(nn.AdaptiveAvgPool2d(1))
            classifier.append(nn.Conv2d(last_bneck_channels, 1280, 1))
            classifier.append(_Hswish(True))
#             classifier.append(nn.Conv2d(1280, nclass, 1))
        elif mode == 'small':
            last_bneck_channels = int(576 * width_mult) if width_mult > 1.0 else 576
            self.layer5 = _ConvBNHswish(self.in_channels, last_bneck_channels, 1, norm_layer=norm_layer)
            classifier.append(SEModule(last_bneck_channels))
            classifier.append(nn.AdaptiveAvgPool2d(1))
            classifier.append(nn.Conv2d(last_bneck_channels, 1280, 1))
            classifier.append(_Hswish(True))
#             classifier.append(nn.Conv2d(1280, nclass, 1))
        else:
            raise ValueError('Unknown mode.')
#         self.classifier = nn.Sequential(*classifier)

        self.flat1280 = nn.Sequential(*classifier)
        n = 1280

        self.lip_color = nn.Linear(n, 3)
        self.eye_color = nn.Linear(n, 3)
        self.hair = nn.Linear(n, 2)
        self.hair_color = nn.Linear(n, 5)
        self.gender = nn.Linear(n, 2)
        self.earring = nn.Linear(n, 2)
        self.smile = nn.Linear(n, 2)
        self.frontal_face = nn.Linear(n, 2)

        self._init_weights()

    def _make_layer(self, block, block_setting, width_mult, dilation=1, norm_layer=nn.BatchNorm2d):
        layers = list()
        for k, exp_size, c, se, nl, s in block_setting:
            out_channels = int(c * width_mult)
            stride = s if (dilation == 1) else 1
            exp_channels = int(exp_size * width_mult)
            layers.append(block(self.in_channels, out_channels, exp_channels, k, stride, dilation, se, nl, norm_layer))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
#         x = self.classifier(x)
        flat1280 = self.flat1280(x)
#         x = x.view(x.size(0), x.size(1))
        
        flat1280 = flat1280.view(flat1280.size(0),flat1280.size(1))

        lip_color = self.lip_color(flat1280)
        eye_color = self.eye_color(flat1280)
        hair = self.hair(flat1280)
        hair_color = self.hair_color(flat1280)
        gender = self.gender(flat1280)
        earring = self.earring(flat1280)
        smile = self.smile(flat1280)
        frontal_face = self.frontal_face(flat1280)

        hair = F.softmax(hair,dim = 1)
        hair_color = F.softmax(hair_color,dim = 1)
        gender = F.softmax(gender,dim = 1)
        earring = F.softmax(earring,dim = 1)
        smile = F.softmax(smile,dim = 1)
        frontal_face = F.softmax(frontal_face,dim = 1)
        
        return [lip_color, eye_color, hair, hair_color, gender, earring, smile, frontal_face]

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

def get_mobilenet_v3(mode='small', width_mult=1.0, pretrained=False, root='~/.torch/models', **kwargs):
    model = MobileNetV3(mode=mode, width_mult=width_mult, **kwargs)
    if pretrained:
        raise ValueError("Not support pretrained")
    return model


def mobilenet_v3_large_1_0(**kwargs):
    return get_mobilenet_v3('large', 1.0, **kwargs)


def mobilenet_v3_small_1_0(**kwargs):
    return get_mobilenet_v3('small', 1.0, **kwargs)


class Loss(torch.nn.Module):
    def __init__(s):
        super().__init__()
        s.MSE = torch.nn.MSELoss()
        s.CE = torch.nn.CrossEntropyLoss()

    def forward(s, preds, colors_b, attrs_b):
#         y = a['lip_color']+a['eye_color']+[a['hair'],a['hair_color'],a['gender'],a['earring'],a['smile'],a['frontal_face']]
        lip_color, eye_color, hair, hair_color, gender, earring, smile, frontal_face = preds
        lpc = s.MSE(lip_color, colors_b[:,:3])
        lc = s.MSE(eye_color, colors_b[:,3:])
        h = s.CE(hair, attrs_b[:, 0])
        hc = s.CE(hair_color, attrs_b[:, 1])
        g = s.CE(gender, attrs_b[:, 2])
        e = s.CE(earring, attrs_b[:, 3])
        sm = s.CE(smile, attrs_b[:, 4])
        f = s.CE(frontal_face, attrs_b[:, 5])
        loss = lpc+lc+h+hc+g+e+sm+f
        
        return loss


### 模型和损失函数说明
模型使用Mobilenetv3的Small版本， 在将最后的输出层更改为8个并行的1x1卷积层，对2个颜色属性进行回归，对6个整型属性进行分类  
回归损失使用MSE，分类损失使用交叉熵损失

In [ ]:
m = mobilenet_v3_small_1_0(nclass=6)
x = torch.rand((2, 3, 250, 250))
y = m(x)
for _ in y:
    print(_.shape)
lip_color, eye_color, hair, hair_color, gender, earring, smile, frontal_face = y
lip_color, eye_color, hair, hair_color, gender, earring, smile, frontal_face

torch.Size([2, 3])
torch.Size([2, 3])
torch.Size([2, 2])
torch.Size([2, 5])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])


(tensor([[-0.8925,  0.1118, -0.1723],
         [-0.1632,  0.2536, -1.3730]], grad_fn=<ViewBackward>),
 tensor([[-0.0142, -0.2719,  0.8314],
         [-2.5911, -0.9223, -0.0742]], grad_fn=<ViewBackward>),
 tensor([[-0.8087, -1.0785],
         [ 0.4037,  1.3462]], grad_fn=<ViewBackward>),
 tensor([[-0.9686, -1.5095,  0.3642, -0.6491, -0.6247],
         [-2.1093, -0.1983, -0.2731, -0.8785, -1.9321]], grad_fn=<ViewBackward>),
 tensor([[-2.1952,  0.4042],
         [-2.5527, -0.1194]], grad_fn=<ViewBackward>),
 tensor([[ 0.1786, -2.2074],
         [ 0.2929, -1.5629]], grad_fn=<ViewBackward>),
 tensor([[-0.4470,  0.1229],
         [-0.4061,  0.9815]], grad_fn=<ViewBackward>),
 tensor([[ 0.8123, -1.0342],
         [ 1.5224, -1.1291]], grad_fn=<ViewBackward>))

In [5]:
def save(savePath, m, epoch, acc):
    d = {
        'param': m.state_dict(),
        'epoch': epoch,
        'acc': acc,
    }
    if isinstance(savePath, Path):
        savePath = savePath.as_posix()
    torch.save(d, savePath)
    print('checkpoint saved as', savePath)

def load(loadPath):
    if isinstance(loadPath, Path):
        loadPath = loadPath.as_posix()
    d = torch.load(loadPath)
    m = Model(mode='small')
    m.load_state_dict(d['param'])
    print('checkpoint loaded from', loadPath)
    e, acc = d['epoch'], d['acc']
    print('epoch:', e, 'acc:', acc)
    return m, d['epoch'], d['acc']

def toCpu(path):
    path = Path(path)
    m, e, acc = load(path)
    m.to(torch.device('cpu'))
    save(path.parents[0]/f'{path.stem}_cpu.ckpt')

In [6]:
def val(em, param, val_dl, d):
    l=Loss()
    vn = len(val_dl.dataset)
    em.load_state_dict(param)
    hair_cnt = 0
    hair_color_cnt = 0
    gender_cnt = 0
    earring_cnt = 0
    smile_cnt = 0
    frontal_face_cnt = 0
    L = 0
    with torch.no_grad():
        for i, (xs, colors_b, attrs_b) in enumerate(val_dl):
            xs, colors_b, attrs_b = xs.to(d), colors_b.to(d), attrs_b.to(d)
            outs = em(xs)
            lip_color, eye_color, hair, hair_color, gender, earring, smile, frontal_face = outs
            L += l(outs, colors_b, attrs_b.long()).item()
            hair_ = torch.max(hair, 1)[1]
            hair_cnt += torch.sum(hair_ == attrs_b[:,0])
            
            hair_color_ = torch.max(hair_color, 1)[1]
            hair_color_cnt += torch.sum(hair_color_ == attrs_b[:,1])
            
            gender_ = torch.max(gender, 1)[1]
            gender_cnt += torch.sum(gender_ == attrs_b[:,2])
            
            earring_ = torch.max(earring, 1)[1]
            earring_cnt += torch.sum(earring_ == attrs_b[:,3])
            
            smile_ = torch.max(smile, 1)[1]
            smile_cnt += torch.sum(smile_ == attrs_b[:,4])
            
            frontal_face_ = torch.max(frontal_face, 1)[1]
            frontal_face_cnt += torch.sum(frontal_face_ == attrs_b[:,5])
    acc = (hair_cnt+hair_color_cnt+gender_cnt+earring_cnt+smile_cnt+frontal_face_cnt)/6/vn
    print(f'validated on {vn} samples| mean acc:{acc*100:.4f}%')
    print(f'hair_cnt:{hair_cnt/vn}|hair_color_cnt:{hair_color_cnt/vn}|gender_cnt:{gender_cnt/vn}')
    print(f'earring_cnt:{earring_cnt/vn}|smile_cnt:{smile_cnt/vn}|frontal_face_cnt:{frontal_face_cnt/vn}')
    print(f'loss:{L/vn:.4f}')
    
    return acc

def train(m,
          d,
          train_dl,
          val_dl,
          saveDir=Path('save'),
          resumePath=None,
          lr=0.001,
          e=50,
          s=10
         ):
    saveDir.mkdir(exist_ok=1)
    startEp = -1
    b = 0
    try:
        m, startEp, b = load(saveDir/'best.ckpt')
    except Exception as err:
        print(err)

    if resumePath is not None:
        m, startEp, b = load(resumePath)

    m.to(d).train()
    em = Model(mode = 'small').to(d).eval()
    
    l=Loss()
    o=torch.optim.SGD(m.parameters(), lr=lr, momentum=0.9)
    
    saveDir.mkdir(exist_ok=1)
    tn = len(train_dl.dataset)
    t = tqdm(range(startEp+1, e))
#     t = range(startEp+1, e)
    for ep in t:
        L = 0
        for i, (xs, colors_b, attrs_b) in enumerate(train_dl):
            xs, colors_b, attrs_b = xs.to(d), colors_b.to(d), attrs_b.to(d)
            o.zero_grad()
            outs = m(xs)
            loss = l(outs, colors_b, attrs_b.long())
            loss.backward()
            o.step()

            L += loss.item()
        t.set_description(f'ep:{ep}| L:{L/tn:.6f}')
        if (ep+1)%s != 0: continue

        acc = val(em, m.state_dict(), val_dl, d)
        save(saveDir/f'{ep:05d}_{acc:.4f}.ckpt', m, ep, acc)
        if b < acc:
            b = acc
            save(saveDir/'best.ckpt', m, ep, acc)
        print(f'E:{ep}| L:{L/tn:.6f}')
    t.close()

In [12]:
d = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(d)
# d = torch.device('cpu')
Model = MobileNetV3
m = Model(mode = 'small').to(d)
train(m,
      d,
      train_dl,
      val_dl,
      saveDir=Path(rootdir + 'save2'),
#       resumePath=Path('save2/03999_0.8062.ckpt'),
      lr=0.0001,
      e=400,
      s=20)

cuda:0
[Errno 2] No such file or directory: '/content/drive/MyDrive/DeepLearningHW/save2/best.ckpt'


ep:19| L:49.853137:   5%|▌         | 20/400 [06:13<55:42,  8.80s/it]

validated on 1046 samples| mean acc:70.2836%
hair_cnt:0.9502868056297302|hair_color_cnt:0.39483746886253357|gender_cnt:0.576481819152832
earring_cnt:0.8212236762046814|smile_cnt:0.6405353546142578|frontal_face_cnt:0.8336520195007324
loss:48.2192
checkpoint saved as /content/drive/MyDrive/DeepLearningHW/save2/00019_0.7028.ckpt
checkpoint saved as /content/drive/MyDrive/DeepLearningHW/save2/best.ckpt
E:19| L:49.853137


ep:39| L:35.990895:  10%|█         | 40/400 [08:43<51:50,  8.64s/it]

validated on 1046 samples| mean acc:70.3952%
hair_cnt:0.9502868056297302|hair_color_cnt:0.3929254412651062|gender_cnt:0.5783938765525818
earring_cnt:0.8212236762046814|smile_cnt:0.6472275257110596|frontal_face_cnt:0.8336520195007324
loss:40.6936
checkpoint saved as /content/drive/MyDrive/DeepLearningHW/save2/00039_0.7040.ckpt
checkpoint saved as /content/drive/MyDrive/DeepLearningHW/save2/best.ckpt
E:39| L:35.990895


...

validated on 1046 samples| mean acc:73.8209%
hair_cnt:0.9502868056297302|hair_color_cnt:0.4034416675567627|gender_cnt:0.7715104818344116
earring_cnt:0.8212236762046814|smile_cnt:0.6491395831108093|frontal_face_cnt:0.8336520195007324
loss:28.0654
checkpoint saved as /content/drive/MyDrive/DeepLearningHW/save2/00359_0.7382.ckpt
E:359| L:4.053710


ep:379| L:5.657294:  95%|█████████▌| 380/400 [52:01<03:01,  9.07s/it]

validated on 1046 samples| mean acc:73.8368%
hair_cnt:0.9502868056297302|hair_color_cnt:0.3967495262622833|gender_cnt:0.7619502544403076
earring_cnt:0.8212236762046814|smile_cnt:0.6663479804992676|frontal_face_cnt:0.8336520195007324
loss:28.6951
checkpoint saved as /content/drive/MyDrive/DeepLearningHW/save2/00379_0.7384.ckpt
E:379| L:5.657294


ep:399| L:5.222742: 100%|██████████| 400/400 [54:36<00:00,  8.19s/it]

validated on 1046 samples| mean acc:73.9165%
hair_cnt:0.9502868056297302|hair_color_cnt:0.41108986735343933|gender_cnt:0.7782026529312134
earring_cnt:0.8212236762046814|smile_cnt:0.6405353546142578|frontal_face_cnt:0.8336520195007324
loss:29.4963
checkpoint saved as /content/drive/MyDrive/DeepLearningHW/save2/00399_0.7392.ckpt
E:399| L:5.222742


In [14]:
d = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
Model = MobileNetV3
em, epo, acc = load(rootdir + 'save2/best.ckpt')
val(Model(mode='small').to(d).eval(), em.state_dict(), val_dl, d)

checkpoint loaded from /content/drive/MyDrive/DeepLearningHW/save2/best.ckpt
epoch: 319 acc: tensor(0.7412, device='cuda:0')
validated on 1046 samples| mean acc:74.1236%
hair_cnt:0.9502868056297302|hair_color_cnt:0.39961758255958557|gender_cnt:0.7801147103309631
earring_cnt:0.8212236762046814|smile_cnt:0.6625239253044128|frontal_face_cnt:0.8336520195007324
loss:28.7963


tensor(0.7412, device='cuda:0')

## result
checkpoint loaded from /content/drive/MyDrive/DeepLearningHW/save2/best.ckpt  
epoch: 319 acc: tensor(0.7412, device='cuda:0')  
validated on 1046 samples| mean acc:74.1236%  
  
hair_cnt:0.9502868056297302  
hair_color_cnt:0.39961758255958557  
gender_cnt:0.7801147103309631  
earring_cnt:0.8212236762046814  
smile_cnt:0.6625239253044128  
frontal_face_cnt:0.8336520195007324  
loss:28.7963  